In [0]:
silver_output_path = "abfss://datalake@stetlprojectdeveus.dfs.core.windows.net/silver/employee/"


In [0]:
spark.conf.set(
  "fs.azure.account.key.stetlprojectdeveus.dfs.core.windows.net",
  dbutils.secrets.get(scope="kv-etl-scope", key="storage-account-key")
)


In [0]:
bronze_input_path = "abfss://datalake@stetlprojectdeveus.dfs.core.windows.net/bronze/"

df_bronze_data = spark.read.format("delta").load(bronze_input_path)

display(df_bronze_data)
df_bronze_data.printSchema()


In [0]:
df_silver = df_bronze_data


In [0]:
df_silver = df_bronze_data.dropDuplicates(["Employee_ID"])


In [0]:
from pyspark.sql.functions import col

df_silver = df_silver.filter((col("Age") >= 16) & (col("Age") <= 70))

In [0]:
from pyspark.sql.functions import when

# Example: If salary is null or less than 0, set it to 0 (or remove row)
df_silver = df_silver.withColumn(
    "Monthly_Salary",
    when((col("Monthly_Salary").isNull()) | (col("Monthly_Salary") < 0), 0)
    .otherwise(col("Monthly_Salary"))
)

In [0]:
from pyspark.sql.functions import lower, trim

df_silver = df_silver.withColumn(
    "Education_Level",
    trim(lower(col("Education_Level")))  # e.g., "high school", "bachelor", "master", "phd"
)

In [0]:
df_silver = df_silver.withColumn(
    "Education_Level",
    when(col("Education_Level") == "high school", "High School")
    .when(col("Education_Level") == "bachelor", "Bachelor")
    .when(col("Education_Level") == "master", "Master")
    .when(col("Education_Level") == "phd", "PhD")
    .otherwise("Other")  # fallback for unknown
)

In [0]:
from pyspark.sql.functions import lit, to_date

cutoff_date = to_date(lit("2025-12-31"))
df_silver = df_silver.withColumn(
    "Hire_Date",
    when(col("Hire_Date") > cutoff_date, lit(None).cast("timestamp"))
    .otherwise(col("Hire_Date"))
)

In [0]:
df_silver = df_silver.withColumnRenamed("Resigned", "HasResigned")


In [0]:
from pyspark.sql.functions import to_date, col

df_silver = df_silver.withColumn(
    "Hire_Date", 
    to_date(col("Hire_Date"))   # or specify a format: to_date(col("Hire_Date"), "yyyy-MM-dd")
)


In [0]:
# review all the transformation
display(df_silver)


In [0]:
silver_output_path = "abfss://datalake@stetlprojectdeveus.dfs.core.windows.net/silver/"

df_silver.write \
    .format("parquet") \
    .mode("overwrite") \
    .save(silver_output_path)
